In [1]:
from brix import Indicator, Handler
import Simulation, US_State
import geopandas as gpd
from shapely.geometry import Point, shape
import json
import requests
import urllib
from geogrid_tools import *

In [2]:
table_name='epa'
geom_type='block_group'

model_radius=5000
reference_radius=2500

- Create the handler
- Get the geogrid
- Identify the state
- Get the state geometry
- find the overlap
- Assign the sim, model, reference zones
- Append the grid "zones" to the actual zones (if the zones properties are updated in one indicator, will they be updated for all?)

- all the above in a world object?

In [3]:
h=Handler(table_name)

In [4]:
# geogrid_data=h.get_geogrid_data(include_geometries=True)

In [5]:

# geogrid=geogrid_data.GEOGRID

In [6]:
# geogrid=h.get_GEOGRID()
# geom = [f['geometry'] for f in geogrid['features']]
# geogrid_gdp=gpd.GeoDataFrame({'geometry':geom})

In [7]:
get_url='https://cityio.media.mit.edu/api/table/'+table_name
with urllib.request.urlopen(get_url+'/GEOGRID') as url:
    geogrid=gpd.read_file(url.read().decode())

centroids=geogrid['geometry'].centroid
geogrid['x_centroid']=[c.x for c in centroids]
geogrid['y_centroid']=[c.y for c in centroids]
geogrid

/tmp/ipykernel_3469565/1794812704.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids=geogrid['geometry'].centroid


,id,interactive,name,geometry,x_centroid,y_centroid
0,0,Web,None,"POLYGON ((-122.14014 37.48867, -122.14014 37.4...",-122.139854,37.488442
1,1,Web,None,"POLYGON ((-122.13957 37.48867, -122.13957 37.4...",-122.139291,37.488442
2,2,Web,None,"POLYGON ((-122.13901 37.48867, -122.13901 37.4...",-122.138727,37.488442
3,3,Web,None,"POLYGON ((-122.13845 37.48867, -122.13845 37.4...",-122.138164,37.488442
4,4,Web,None,"POLYGON ((-122.13788 37.48867, -122.13788 37.4...",-122.137600,37.488442
...,...,...,...,...,...,...
745,745,Web,None,"POLYGON ((-122.12887 37.47564, -122.12887 37.4...",-122.128586,37.475420
746,746,Web,None,"POLYGON ((-122.12830 37.47564, -122.12830 37.4...",-122.128022,37.475420
747,747,Web,None,"POLYGON ((-122.12774 37.47564, -122.12774 37.4...",-122.127459,37.475420
748,748,Web,None,"POLYGON ((-122.12718 37.47564, -122.12718 37.4...",-122.126896,37.475420


In [8]:
types=json.load(open('./data/default_types.json'))

In [9]:
try:
    interactive_zone=gpd.read_file('./tables/{}/interactive.geojson'.format(table_name))
    print('using interactive zone')
except:
    interactive_zone=None
properties=init_geogrid(table_name, interactive_zone=interactive_zone, types=types)

Initialise GEOGRID: <Response [200]>


In [10]:
state_fips=identify_state(properties)

In [ ]:
st=US_State.US_State(state_fips, year=2018, geom_type=geom_type)
st.get_geometry()
# st.remove_non_urban_zones()
st.get_lodes_data( include=['wac', 'rac', 'od'])
st.add_lodes_cols_to_shape()
st.geom.plot()

Getting geometry (block_group) for state: California


/home/doorleyr/CS_Spatial_Modules/US_State.py:54: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids=self.geom['geometry'].centroid


Getting WAC data from https://lehd.ces.census.gov/data/lodes/LODES7/
	 Formatting WAC data
Getting RAC data from https://lehd.ces.census.gov/data/lodes/LODES7/
	 Formatting RAC data
Getting OD data from https://lehd.ces.census.gov/data/lodes/LODES7/
	 Formatting OD data


In [ ]:
st.subset_geom_by_distance(centre_x_y=[geogrid.x_centroid.mean(), geogrid.y_centroid.mean()], 
                                   radius=reference_radius, name='reference_area')

st.subset_geom_by_distance(centre_x_y=[geogrid.x_centroid.mean(), geogrid.y_centroid.mean()], 
                                   radius=model_radius, name='model_area')

st.geom=assign_sim_area(gpd.GeoDataFrame(geogrid), st.geom)

In [ ]:
zones=st.geom.rename(columns={col: 'area' for col in st.geom.columns if 'ALAND' in col})

In [ ]:
zones.loc[zones['model_area']].to_file('tables/{}/zones.geojson'.format(table_name), driver='GeoJSON')

In [ ]:
import os
if not os.path.isdir('tables/{}'.format(table_name)):
    os.mkdir('tables/{}'.format(table_name))

In [ ]:
simpop_df=st.lodes_to_pop_table(
    model_subset_name='model_area', sim_subset_name='sim_area')
simpop_df.to_csv('tables/{}/simpop_df.csv'.format(table_name), index=False)

In [ ]:
geogrid.to_file('tables/{}/geogrid.geojson'.format(table_name), driver='GeoJSON')